In [ ]:
import json
import os

from datetime import datetime
import pandas as pd
import nltk

import matplotlib.pyplot as plt

In [ ]:
messages_path = 'messages/'

In [ ]:
df = pd.DataFrame()
for file in os.listdir(messages_path):
    data = json.load(open(os.path.join(messages_path,file),"r"))
    df = df.append(pd.json_normalize(data['messages']))
df = df.rename(columns={'timestamp_ms':'datetime'})
df['datetime'] = df.apply(lambda row: datetime.fromtimestamp(row['datetime']/1000),axis=1)
df = df[df["type"]=="Generic"]
df.sort_values(by='datetime',inplace=True)
df = df.reset_index()
df

In [ ]:
per_day = df.groupby(pd.Grouper(freq='D',key='datetime')).count()
per_day['index'].plot.line()
plt.title("Messages per Day")
plt.xlabel("Time")
plt.ylabel("Messages")
plt.show()

In [ ]:
words = df[df.content.notnull()]["content"]
pd.Series(' '.join(words).lower().split()).value_counts()[:20].plot.bar()
plt.show()

In [ ]:
words = [w for w in ' '.join(words).lower().split() if w.isalpha()]

In [ ]:
stopwords = nltk.corpus.stopwords.words("english")
words = [w for w in words if w not in stopwords]
fd = nltk.FreqDist(words)
fd

In [ ]:
finder = nltk.collocations.TrigramCollocationFinder.from_words(words)
finder.ngram_fd.most_common(30)

In [ ]:
from nltk.sentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()
df["sentiment"] = df[df.content.notnull()]["content"].apply(lambda content: sia.polarity_scores(content))